In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, current_timestamp

container_name = "bronze"
storage_account_name = "coindeckostorage"

# SAS token
sas_token = "sp=r&st=2026-02-15T04:33:59Z&se=2026-06-11T12:48:59Z&spr=https&sv=2024-11-04&sr=c&sig=bhRbI4CleieMszi6bpxADgAhKW5XScyDWz0oxuG%2FGHA%3D"

# Configure Spark to use SAS token
spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
    sas_token
)

# Define the file path (wasbs for SAS)
file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/data.json"

# Read JSON file into DataFrame
bronze_df = spark.read.json(file_path)


In [0]:
from pyspark.sql.functions import col, to_timestamp

# Flatten ROI struct and cast date fields
silver_df = bronze_df \
    .withColumn("roi_currency", col("roi.currency")) \
    .withColumn("roi_percentage", col("roi.percentage")) \
    .withColumn("roi_times", col("roi.times")) \
    .withColumn("ath_date", to_timestamp(col("ath_date"))) \
    .withColumn("atl_date", to_timestamp(col("atl_date"))) \
    .withColumn("last_updated", to_timestamp(col("last_updated")))

# Select only necessary columns for Silver layer
silver_df = silver_df.select(
    col("id").alias("coin_id"),
    col("name").alias("coin_name"),
    col("symbol"),
    col("current_price"),
    col("market_cap"),
    col("total_volume"),
    col("high_24h"),
    col("low_24h"),
    col("ath"),
    col("ath_change_percentage"),
    col("ath_date"),
    col("atl"),
    col("atl_change_percentage"),
    col("atl_date"),
    col("circulating_supply"),
    col("total_supply"),
    col("max_supply"),
    col("fully_diluted_valuation"),
    col("market_cap_rank"),
    col("market_cap_change_24h"),
    col("market_cap_change_percentage_24h"),
    col("price_change_24h"),
    col("price_change_percentage_24h"),
    col("roi_currency"),
    col("roi_percentage"),
    col("roi_times"),
    col("last_updated")
)


coin_id,coin_name,symbol,current_price,market_cap,total_volume,high_24h,low_24h,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,circulating_supply,total_supply,max_supply,fully_diluted_valuation,market_cap_rank,market_cap_change_24h,market_cap_change_percentage_24h,price_change_24h,price_change_percentage_24h,roi_currency,roi_percentage,roi_times,last_updated
bitcoin,Bitcoin,btc,69695.0,1393138460404,4.0377815454E10,70434.0,68713.0,126080.0,-44.72178,2025-10-06T18:57:42.558Z,67.81,102680.93343,2013-07-06T00:00:00Z,1.9989271E7,1.9989293E7,2.1E7,1393139993679,1,1.1983924861E10,0.86767,629.64,0.91166,null,null,null,2026-02-14T17:50:37.897Z
ethereum,Ethereum,eth,2079.35,250960559298,1.6585382349E10,2100.05,2042.64,4946.05,-57.95929,2025-08-24T19:21:03.333Z,0.432979,480143.72949,2015-10-20T00:00:00Z,1.206924676142904E8,1.206924676142904E8,null,250960559298,2,2.320801408E9,0.9334,20.13,0.97743,btc,3888.375137594402,38.88375137594402,2026-02-14T17:50:37.561Z
tether,Tether,usdt,0.99965,183752815016,6.4611390117E10,0.999905,0.999412,1.32,-24.44831,2018-07-24T00:00:00Z,0.572521,74.59986,2015-03-02T00:00:00Z,1.838169601286833E11,1.892838973700162E11,null,189217844505,3,-4.942387838537598E7,-0.02689,1.7129E-4,0.01714,null,null,null,2026-02-14T17:50:36.369Z
ripple,XRP,xrp,1.47,89768349733,2.208102175E9,1.48,1.4,3.65,-59.58747,2025-07-18T03:40:53.808Z,0.00268621,54757.78879,2014-05-22T00:00:00Z,6.0917315351E10,9.9985721048E10,1.0E11,147339933214,4,3.363785348E9,3.89306,0.056924,4.01818,null,null,null,2026-02-14T17:50:36.282Z
binancecoin,BNB,bnb,631.7,86137453641,8.68964406E8,635.98,614.36,1369.99,-53.89037,2025-10-13T08:41:24.131Z,0.0398177,1586375.69989,2017-10-19T00:00:00Z,1.3635932226E8,1.3635932069E8,2.0E8,86137452649,5,2.086871937E9,2.48288,15.35,2.49102,null,null,null,2026-02-14T17:50:37.499Z
usd-coin,USDC,usdc,0.999904,73553841883,7.308414375E9,1.0,0.999818,1.17,-14.73869,2019-05-08T00:40:28.3Z,0.877647,13.92565,2023-03-11T08:02:13.981Z,7.35609048560082E10,7.355965342921922E10,null,73552590576,6,5.11699788E8,0.70055,-1.4361288290354E-5,-0.00144,null,null,null,2026-02-14T17:50:36.3Z
solana,Solana,sol,87.96,49958020667,3.217487643E9,88.22,84.22,293.31,-70.01117,2025-01-19T11:15:27.957Z,0.500801,17464.01909,2020-05-11T19:35:23.449Z,5.679581924596868E8,6.203917611709459E8,null,54570115966,7,2.078210558E9,4.34047,3.64,4.321,null,null,null,2026-02-14T17:50:36.104Z
tron,TRON,trx,0.283031,26809400784,5.26999131E8,0.283713,0.279858,0.431288,-34.37557,2024-12-04T00:10:40.323Z,0.00180434,15586.0819,2017-11-12T00:00:00Z,9.472193777640399E10,9.47232250498033E10,null,26809765124,8,2.80827787E8,1.05859,0.00301815,1.07786,usd,14796.342448076179,147.9634244807618,2026-02-14T17:50:36.102Z
dogecoin,Dogecoin,doge,0.105305,17766059542,1.070030688E9,0.105515,0.096004,0.731578,-85.60577,2021-05-08T05:08:23.458Z,8.69E-5,121074.34051,2015-05-06T00:00:00Z,1.68726213126579E11,1.687395731265791E11,null,17767466286,9,1.46985574E9,9.01962,0.00879101,9.10853,null,null,null,2026-02-14T17:50:36.101Z
figure-heloc,Figure Heloc,figr_heloc,1.033,15657687496,1.1168116E7,1.042,1.033,1.37,-24.59084,2025-10-28T06:05:38.345Z,0.155357,564.79712,2025-10-31T14:27:24.456Z,1.5160334529097E10,1.5160334529097E10,null,15657687496,10,-1.0235402761528206E8,-0.64945,-0.007863799999999976,-0.75565,null,null,null,2026-02-14T17:50:35.773Z


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, current_timestamp

# Handle nulls with defaults
silver_df = silver_df.fillna({
    "market_cap": 0,
    "current_price": 0,
    "total_volume": 0,
    "circulating_supply": 0,
    "total_supply": 0,
    "max_supply": 0,
    "roi_percentage": 0,
    "roi_times": 0
})

# Filter out invalid rows
silver_df = silver_df.filter(col("id").isNotNull() & col("name").isNotNull())

# Normalize column names (lowercase + snake_case)
for col_name in silver_df.columns:
    silver_df = silver_df.withColumnRenamed(col_name, col_name.lower().replace(" ", "_"))



coin_id,coin_name,symbol,current_price,market_cap,total_volume,high_24h,low_24h,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,circulating_supply,total_supply,max_supply,fully_diluted_valuation,market_cap_rank,market_cap_change_24h,market_cap_change_percentage_24h,price_change_24h,price_change_percentage_24h,roi_currency,roi_percentage,roi_times,last_updated
bitcoin,Bitcoin,btc,69695.0,1393138460404,4.0377815454E10,70434.0,68713.0,126080.0,-44.72178,2025-10-06T18:57:42.558Z,67.81,102680.93343,2013-07-06T00:00:00Z,1.9989271E7,1.9989293E7,2.1E7,1393139993679,1,1.1983924861E10,0.86767,629.64,0.91166,null,0.0,0.0,2026-02-14T17:50:37.897Z
ethereum,Ethereum,eth,2079.35,250960559298,1.6585382349E10,2100.05,2042.64,4946.05,-57.95929,2025-08-24T19:21:03.333Z,0.432979,480143.72949,2015-10-20T00:00:00Z,1.206924676142904E8,1.206924676142904E8,0.0,250960559298,2,2.320801408E9,0.9334,20.13,0.97743,btc,3888.375137594402,38.88375137594402,2026-02-14T17:50:37.561Z
tether,Tether,usdt,0.99965,183752815016,6.4611390117E10,0.999905,0.999412,1.32,-24.44831,2018-07-24T00:00:00Z,0.572521,74.59986,2015-03-02T00:00:00Z,1.838169601286833E11,1.892838973700162E11,0.0,189217844505,3,-4.942387838537598E7,-0.02689,1.7129E-4,0.01714,null,0.0,0.0,2026-02-14T17:50:36.369Z
ripple,XRP,xrp,1.47,89768349733,2.208102175E9,1.48,1.4,3.65,-59.58747,2025-07-18T03:40:53.808Z,0.00268621,54757.78879,2014-05-22T00:00:00Z,6.0917315351E10,9.9985721048E10,1.0E11,147339933214,4,3.363785348E9,3.89306,0.056924,4.01818,null,0.0,0.0,2026-02-14T17:50:36.282Z
binancecoin,BNB,bnb,631.7,86137453641,8.68964406E8,635.98,614.36,1369.99,-53.89037,2025-10-13T08:41:24.131Z,0.0398177,1586375.69989,2017-10-19T00:00:00Z,1.3635932226E8,1.3635932069E8,2.0E8,86137452649,5,2.086871937E9,2.48288,15.35,2.49102,null,0.0,0.0,2026-02-14T17:50:37.499Z
usd-coin,USDC,usdc,0.999904,73553841883,7.308414375E9,1.0,0.999818,1.17,-14.73869,2019-05-08T00:40:28.3Z,0.877647,13.92565,2023-03-11T08:02:13.981Z,7.35609048560082E10,7.355965342921922E10,0.0,73552590576,6,5.11699788E8,0.70055,-1.4361288290354E-5,-0.00144,null,0.0,0.0,2026-02-14T17:50:36.3Z
solana,Solana,sol,87.96,49958020667,3.217487643E9,88.22,84.22,293.31,-70.01117,2025-01-19T11:15:27.957Z,0.500801,17464.01909,2020-05-11T19:35:23.449Z,5.679581924596868E8,6.203917611709459E8,0.0,54570115966,7,2.078210558E9,4.34047,3.64,4.321,null,0.0,0.0,2026-02-14T17:50:36.104Z
tron,TRON,trx,0.283031,26809400784,5.26999131E8,0.283713,0.279858,0.431288,-34.37557,2024-12-04T00:10:40.323Z,0.00180434,15586.0819,2017-11-12T00:00:00Z,9.472193777640399E10,9.47232250498033E10,0.0,26809765124,8,2.80827787E8,1.05859,0.00301815,1.07786,usd,14796.342448076179,147.9634244807618,2026-02-14T17:50:36.102Z
dogecoin,Dogecoin,doge,0.105305,17766059542,1.070030688E9,0.105515,0.096004,0.731578,-85.60577,2021-05-08T05:08:23.458Z,8.69E-5,121074.34051,2015-05-06T00:00:00Z,1.68726213126579E11,1.687395731265791E11,0.0,17767466286,9,1.46985574E9,9.01962,0.00879101,9.10853,null,0.0,0.0,2026-02-14T17:50:36.101Z
figure-heloc,Figure Heloc,figr_heloc,1.033,15657687496,1.1168116E7,1.042,1.033,1.37,-24.59084,2025-10-28T06:05:38.345Z,0.155357,564.79712,2025-10-31T14:27:24.456Z,1.5160334529097E10,1.5160334529097E10,0.0,15657687496,10,-1.0235402761528206E8,-0.64945,-0.007863799999999976,-0.75565,null,0.0,0.0,2026-02-14T17:50:35.773Z


In [0]:

# Storage details
container_name = "silver"
storage_account_name = "coindeckostorage"

# SAS token
sas_token = "sp=racwdlmeop&st=2026-02-15T04:13:54Z&se=2026-06-18T12:28:54Z&spr=https&sv=2024-11-04&sr=c&sig=jzBah6PxChRvUBDlnKJLxfSCjvEx5U0dGwCIYD80xB0%3D"

# Configure Spark to use SAS token
spark.conf.set(
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net",
    sas_token
)

# ---- Write Silver DataFrame to ADLS as Delta using SAS ----
silver_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/cleanedData/"

silver_df.write.format("delta").mode("overwrite").partitionBy("coin_id").save(silver_path)


coin_id,coin_name,symbol,current_price,market_cap,total_volume,high_24h,low_24h,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,circulating_supply,total_supply,max_supply,fully_diluted_valuation,market_cap_rank,market_cap_change_24h,market_cap_change_percentage_24h,price_change_24h,price_change_percentage_24h,roi_currency,roi_percentage,roi_times,last_updated
bitcoin,Bitcoin,btc,69695.0,1393138460404,4.0377815454E10,70434.0,68713.0,126080.0,-44.72178,2025-10-06T18:57:42.558Z,67.81,102680.93343,2013-07-06T00:00:00Z,1.9989271E7,1.9989293E7,2.1E7,1393139993679,1,1.1983924861E10,0.86767,629.64,0.91166,null,0.0,0.0,2026-02-14T17:50:37.897Z
ethereum,Ethereum,eth,2079.35,250960559298,1.6585382349E10,2100.05,2042.64,4946.05,-57.95929,2025-08-24T19:21:03.333Z,0.432979,480143.72949,2015-10-20T00:00:00Z,1.206924676142904E8,1.206924676142904E8,0.0,250960559298,2,2.320801408E9,0.9334,20.13,0.97743,btc,3888.375137594402,38.88375137594402,2026-02-14T17:50:37.561Z
tether,Tether,usdt,0.99965,183752815016,6.4611390117E10,0.999905,0.999412,1.32,-24.44831,2018-07-24T00:00:00Z,0.572521,74.59986,2015-03-02T00:00:00Z,1.838169601286833E11,1.892838973700162E11,0.0,189217844505,3,-4.942387838537598E7,-0.02689,1.7129E-4,0.01714,null,0.0,0.0,2026-02-14T17:50:36.369Z
ripple,XRP,xrp,1.47,89768349733,2.208102175E9,1.48,1.4,3.65,-59.58747,2025-07-18T03:40:53.808Z,0.00268621,54757.78879,2014-05-22T00:00:00Z,6.0917315351E10,9.9985721048E10,1.0E11,147339933214,4,3.363785348E9,3.89306,0.056924,4.01818,null,0.0,0.0,2026-02-14T17:50:36.282Z
binancecoin,BNB,bnb,631.7,86137453641,8.68964406E8,635.98,614.36,1369.99,-53.89037,2025-10-13T08:41:24.131Z,0.0398177,1586375.69989,2017-10-19T00:00:00Z,1.3635932226E8,1.3635932069E8,2.0E8,86137452649,5,2.086871937E9,2.48288,15.35,2.49102,null,0.0,0.0,2026-02-14T17:50:37.499Z
usd-coin,USDC,usdc,0.999904,73553841883,7.308414375E9,1.0,0.999818,1.17,-14.73869,2019-05-08T00:40:28.3Z,0.877647,13.92565,2023-03-11T08:02:13.981Z,7.35609048560082E10,7.355965342921922E10,0.0,73552590576,6,5.11699788E8,0.70055,-1.4361288290354E-5,-0.00144,null,0.0,0.0,2026-02-14T17:50:36.3Z
solana,Solana,sol,87.96,49958020667,3.217487643E9,88.22,84.22,293.31,-70.01117,2025-01-19T11:15:27.957Z,0.500801,17464.01909,2020-05-11T19:35:23.449Z,5.679581924596868E8,6.203917611709459E8,0.0,54570115966,7,2.078210558E9,4.34047,3.64,4.321,null,0.0,0.0,2026-02-14T17:50:36.104Z
tron,TRON,trx,0.283031,26809400784,5.26999131E8,0.283713,0.279858,0.431288,-34.37557,2024-12-04T00:10:40.323Z,0.00180434,15586.0819,2017-11-12T00:00:00Z,9.472193777640399E10,9.47232250498033E10,0.0,26809765124,8,2.80827787E8,1.05859,0.00301815,1.07786,usd,14796.342448076179,147.9634244807618,2026-02-14T17:50:36.102Z
dogecoin,Dogecoin,doge,0.105305,17766059542,1.070030688E9,0.105515,0.096004,0.731578,-85.60577,2021-05-08T05:08:23.458Z,8.69E-5,121074.34051,2015-05-06T00:00:00Z,1.68726213126579E11,1.687395731265791E11,0.0,17767466286,9,1.46985574E9,9.01962,0.00879101,9.10853,null,0.0,0.0,2026-02-14T17:50:36.101Z
figure-heloc,Figure Heloc,figr_heloc,1.033,15657687496,1.1168116E7,1.042,1.033,1.37,-24.59084,2025-10-28T06:05:38.345Z,0.155357,564.79712,2025-10-31T14:27:24.456Z,1.5160334529097E10,1.5160334529097E10,0.0,15657687496,10,-1.0235402761528206E8,-0.64945,-0.007863799999999976,-0.75565,null,0.0,0.0,2026-02-14T17:50:35.773Z
